In [2]:
import time
import requests 
import pandas as pd 

**Used Copilot to help construct the following functions**

In [3]:
def get_steam_reviews(appid):
    """
    Fetches game reviews from Steam API for a given appid.
    
    Parameters:
    appid (int): The Steam AppID of the game
    
    Returns:
    tuple: (games_df)
        - games_df: DataFrame containing individual reviews
    """
    # Construct API URL
    url = f'http://store.steampowered.com/appreviews/{appid}?json=1'
    
    try:
        # Get response from API
        response = requests.get(url).json()
        games = response['reviews']
        games_summary = response['query_summary']
        
        # Process individual reviews
        game_data = []
        for game in games:
            game_data.append({
                'game_id': str(appid),  # Using appid as game_title since we don't fetch the title
                'recommendationid': game['recommendationid'],
                'review': game['review'],
            })
            time.sleep(1)
        
        
        
        # Create DataFrames
        games_df = pd.DataFrame(game_data)
        
        return games_df
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

In [4]:
def get_steam_summaries(appid):
    """
    Fetches summary statistics from Steam API for a given appid.
    
    Parameters:
    appid (int): The Steam AppID of the game
    
    Returns:
    tuple: (games_summary_df)
        - games_summary_df: DataFrame containing summary statistics
    """
    # Construct API URL
    url = f'http://store.steampowered.com/appreviews/{appid}?json=1'
    
    try:
        # Get response from API
        response = requests.get(url).json()
        games = response['reviews']
        games_summary = response['query_summary'] 

        game_summary_data = [{
            'game_id': str(appid),
            'total_reviews': games_summary['total_reviews'],
            'total_positive': games_summary['total_positive'],
            'total_negative': games_summary['total_negative'],
            'review_score': games_summary['review_score'],
            'review_score_desc': games_summary['review_score_desc'],
        }]
        time.sleep(1)
        
        # Create DataFrames
        games_summary_df = pd.DataFrame(game_summary_data)
        
        return games_summary_df
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

In [5]:
from tqdm import tqdm

def get_multiple_steam_reviews(appid_list):
    """
    Fetches reviews for multiple Steam games and combines them into a single DataFrame.
    
    Parameters:
    appid_list (list): List of Steam AppIDs
    
    Returns:
    DataFrame: Combined DataFrame containing reviews for all games
    """
    # Initialize an empty list to store all game reviews
    all_reviews = []
    
    # Process each game ID with a progress bar
    for appid in tqdm(appid_list, desc="Processing games"):
        # Get reviews for current game
        game_reviews = get_steam_reviews(appid)
        
        if game_reviews is not None:
            # Add to our collection
            all_reviews.append(game_reviews)
            
            # Add a delay to be nice to Steam's API
            time.sleep(2)
        else:
            print(f"Skipping AppID {appid} due to error")
    
    # Combine all reviews into a single DataFrame
    if all_reviews:
        combined_df = pd.concat(all_reviews, ignore_index=True)
        return combined_df
    else:
        return pd.DataFrame()  # Return empty DataFrame if no reviews were collected

In [7]:
#Example with the game Rust(id: 252490) 
get_steam_summaries(252490)

,game_id,total_reviews,total_positive,total_negative,review_score,review_score_desc
0,252490,464342,404875,59467,8,Very Positive


In [6]:
#example with the game Rust(id: 252490)
get_steam_reviews(252490)

,game_id,recommendationid,review
0,252490,177409644,Rust is a great game. It is like real life if ...
1,252490,177844978,[h1]My [i]Rust[/i] Review - 223 Hours In[/h1]\...
2,252490,177619447,This game makes me want to kill myself. It's s...
3,252490,178793028,"Great game very unique and grindy, which I per..."
4,252490,178578421,"Great fun, very addictive and highly toxic... ..."
5,252490,177383935,game is horrible. toxic community and the supp...
6,252490,177862501,fucking ass. wont let me in the game because i...
7,252490,178818993,This game has ruined my life. Every aspect of ...
8,252490,178649300,bless up all staging branch nexus system alpha...
9,252490,177563743,its the most unfair game and company i have ev...


In [ ]:
game_ids = []
reviews_df = get_multiple_steam_reviews(game_ids) 
reviews_df.to_csv('steam_reviews.csv', index=False)

NameError: name 'game_ids' is not defined